In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
import stisim as ss
import stisim.utils as ssu
import numpy as np
import pandas as pd

IDEA 1: Draw more random numbers than needed on each time step and use by row of people arrays.
Set seed to seed + ti on each step?

In [52]:
ppl = ss.People(10)
ppl.networks = ss.ndict(ss.mf(), ss.maternal())

hiv = ss.HIV()
hiv.pars['beta'] = {'mf': [0.0008, 0.0004], 'maternal': [0.2, 0]}

sim = ss.Sim(people=ppl, modules=[hiv, ss.Gonorrhea(), ss.Pregnancy()])
sim.initialize();
#sim.run()

Initializing sim (warning: not resetting sim.people) with 10000 agents


In [53]:
def dfp(vec):
    display(pd.DataFrame({'result': vec}, index=pd.Index(sim.people.uid, name='uid')))

In [54]:
ssu.set_seed(0)
dfp(np.random.binomial(n=1, p=len(sim.people)*[0.3]))
dfp(np.random.choice([True, False], len(sim.people)))

,result
uid,
0,0
1,1
2,0
3,0
4,0
5,0
6,0
7,1
8,1


,result
uid,
0,True
1,False
2,False
3,True
4,True
5,False
6,False
7,False
8,False


In [55]:
# Check that results are repeatable
ssu.set_seed(0)
dfp(np.random.binomial(n=1, p=len(sim.people)*[0.3]))
dfp(np.random.choice([True, False], len(sim.people)))

,result
uid,
0,0
1,1
2,0
3,0
4,0
5,0
6,0
7,1
8,1


,result
uid,
0,True
1,False
2,False
3,True
4,True
5,False
6,False
7,False
8,False


In [56]:
# Now remove agent with uid=0
sim.people.remove(0)
sim.people.uid

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [57]:
ssu.set_seed(0)
dfp(np.random.binomial(n=1, p=len(sim.people)*[0.3]))
dfp(np.random.choice([True, False], len(sim.people)))

,result
uid,
1,0
2,1
3,0
4,0
5,0
6,0
7,0
8,1
9,1


,result
uid,
1,True
2,False
3,True
4,False
5,False
6,True
7,True
8,False
9,False


In [58]:
# Now try sampling for all uids and keeping only uids in the current view
ssu.set_seed(0)
dfp(np.random.binomial(n=1, p=len(sim.people._uid_map)*[0.3])[sim.people.uid])
dfp(np.random.choice([True, False], len(sim.people._uid_map))[sim.people.uid])

# ^^^ WORKS! (These are independent draws per-individual)

,result
uid,
1,1
2,0
3,0
4,0
5,0
6,0
7,1
8,1
9,0


,result
uid,
1,False
2,False
3,True
4,True
5,False
6,False
7,False
8,False
9,True


Now working through utils towards integration

In [59]:
ssu.sample(dist='normal', par1=10, par2=1, size=len(sim.people))

array([10.48431215, 10.57914048,  9.81841743, 11.41020463,  9.62552831,
       10.27519832,  9.03924539, 10.37692697, 10.03343893])

In [60]:
ssu.set_seed(0)
# Choice (potentially) adds correlation across the entire population
# The following is fine, because the draws are independent per agent
np.random.choice([True, False], size=5)

# But choosing k of n agents is a centralized thing, right?
# Internally, this must use just two random numbers, not one per agent
np.random.choice(sim.people.uid, 2, replace=False)

# Centralized choices in the model:
# * Choosing k initial cases (could be a prob per agent instead?)
# * Choosing k agents to start/stop ART
# ~~~ Very few! Just need to encourage programming away from this pattern.

array([7, 6])

# Play with random number streams

In [61]:
r1 = np.random.default_rng(seed=0)
display(r1.random(5))

r2 = np.random.default_rng(seed=1)
display(r2.random(5))

r1 = np.random.default_rng(seed=0)
r2 = np.random.default_rng(seed=1)
for i in range(5):
    print(r1.random())
print()
for i in range(5):
    print(r2.random())

r1 = np.random.default_rng(seed=0)
r2 = np.random.default_rng(seed=1)
for i in range(5):
    print()
    print(r1.random())
    print(r2.random())

# So! Multiple random number streams are easy and work as expected.

array([0.63696169, 0.26978671, 0.04097352, 0.01652764, 0.81327024])

array([0.51182162, 0.9504637 , 0.14415961, 0.94864945, 0.31183145])

0.6369616873214543
0.2697867137638703
0.04097352393619469
0.016527635528529094
0.8132702392002724

0.5118216247002567
0.9504636963259353
0.14415961271963373
0.9486494471372439
0.31183145201048545

0.6369616873214543
0.5118216247002567

0.2697867137638703
0.9504636963259353

0.04097352393619469
0.14415961271963373

0.016527635528529094
0.9486494471372439

0.8132702392002724
0.31183145201048545


In [125]:
r1

Generator(PCG64) at 0x17E474740

# Let's do some performance evaluation

In [83]:
# k=1 person in a population of n=100000 get infected. What difference does it make if we draw k vs n?
r1 = np.random.default_rng(seed=0)

k = 100
n = 100_000
trials = 100
import sciris as sc

t = sc.timer('Draw k')
for i in range(trials):
    v = r1.random(size=k)
t.toc()

t = sc.timer('Draw n, keep k')
for i in range(trials):
    v = r1.random(size=n)[:k]
t.toc()

# UGH, much slower, but still doesn't take all that much time compared to other calculations
# Sample things like prognosis (once) for each agent at creation...
# Okay for S->I... except not: in HIV, prognosis depends on age and possibly other factors
# Plus pure burn if nobody infected on a given step

Draw k: 0.316 ms
Draw n, keep k: 43.8 ms


In [121]:
# What about separate generators per-agent?

r = [np.random.default_rng(seed=s) for s in range(n)]
t = sc.timer('Draw k from separate streams [list comprehension]')
for i in range(trials):
    v = [z.choice([True, False]) for i,z in enumerate(r) if i < k]
t.toc()

r = [np.random.default_rng(seed=s) for s in range(n)]
t = sc.timer('Draw k from separate streams [map]')
for i in range(trials):
    #v = list(map(lambda x: x.random(), r[:k]))
    v = list(map(lambda x: x.choice([True, False]), r[:k]))
t.toc()

# Map is much faster!

Draw k from separate streams [list comprehension]: 0.460 s
Draw k from separate streams [map]: 44.1 ms


# Binomial filtering

In [106]:
denom_conds = sim.people.female
uids_to_choose_from = ss.true(denom_conds)

ssu.set_seed(0)
# Uses len(arr) random numbers:
u1 = ss.bernoulli_filter(0.7, uids_to_choose_from) # return arr[(np.random.random(len(arr)) < prob).nonzero()[0]]
display(u1)

r1 = np.random.default_rng(seed=0)
def bernoulli_filter(prob, arr):
    # Uses len(uid map) random numbers
    return arr[(r1.random(len(sim.people._uid_map)) < prob)[arr].nonzero()[0]]

u2 = bernoulli_filter(0.7, uids_to_choose_from)
display(u2)

array([1, 8])

array([1, 8])

## Point is greater comparability between two simulations.

Can imagine all this working until the point at which the uid map grows in one simulation (baseline) but not another (intervention). Resetting the seed each timestep helps, if/when the (intervention) sim gets a similar uid growth. Oh but this is assuming the uids grow in big chunks. Probably need to be drawing enough random numbers for each decision to cover all (live) agents in a consistent manner.

Need to draw not len(uid map) but rather len(people ever in simulation)

NEED: equivalent of one stream per agent-decision-timestep

Above concept uses "slots" where a agent-decision indexes into a block of rngs per timestep. Problem is that blocksize is tied to population size, which will cause offsets between simulations even with per-step seed resetting.

* Fixed block size is possible, but inefficient.
* Can't use current population size because sub-blocks are selected from one RNG sequentially
* [ --SUBBLOCK_1-- ][ --SUBBLOCK_2-- ]...[ --SUBBLOCK_N-- ]
* If length of subblock (e.g. number of uids) grows, then start of next subblock is off
* --> Use different rngs for each SUBBLOCK, or fix seed at beginning of each SUBBLOCK?!
* --> Still need to draw full uid size and sample within so that decisions correctly map to agents
* --> Is RNG resetting fast?

* At some point, per-agent RNG streams with slots per decision-timestep becomes more efficient

In [119]:
t = sc.timer('set_seed')
for i in range(100_000):
    ssu.set_seed(i)
t.toc()
# Seems plenty fast

set_seed: 0.175 s


grow() happens early in a timestep, so
* TIME0 with pop N0-->N1: [seed 0, draw N0][seed 1, draw N0]...[seed d, draw N0][seed d+1, draw N1][seed d+2, draw N1]...[seed D, draw N1]
* Same for time 1 with seeds offset by a fixed amount more than D
* Works because the len of the uid_map (N0, N1, ...) only grows as new agents are added at the end of the population arrays
* Long running simulations, at the end, will still be drawing numbers for long-since-dead/removed agents - inefficient, but fixed problem of needing to know the final total number of agents

What if timestep flow conditionally adds or removes a decision?
* Each decision could have a unique offset instead of going sequentially
* Offset per-timestep would be max of unique within-timestep offsets

Use case is comparing two simulations with different networks, A vs B. The networks will use random numbers in forming partners, and each can have their own offset so that decisions that come later in the flow would not be affected by the network's use of random numbers. There WOULD be differences due to the different pairs created, however.
* If edge 1-2 exists in both A and B, being consistent about transmission on this edge between sims would be tricky.
* Could sample transmission from approx N^2 (per layer?!) (N*(N-1)/2 if heterosexual) and select down to k active edges...
* Do network layers get "flattened" before transmission?
* If transmission is not consistent... what's the point
* Any deviation in the network throws everything off?

* Confirmed uses length(edges) random numbers for p1-->p2 and again length(edges) for p2-->p1

* OR!!! Reset seed on a per-edge basis in some clever way? That would require one call per edge instead of a length(edges) vectorized call.
* Make an RNG per-edge for transmission related inquiries, seeded based on node properties?

Switch to node-based view?
* Node i connected to k other nodes
* k chances to get infected on this step
* Combine probability of infection to a single RNG draw
* Use one slot per agent
* Do I get infected on this timestep?
* If yes, need to determine from whom - sample 1 of k proportional to prob, roulette --> or keep list of possible sources and tree later?
* NOTE also for each pathogen - so part of module

In [110]:
print(len(sim.people))
print(len(sim.people._uid_map))

9
10


In [111]:
sim.people.grow(3)

array([10, 11, 12])

In [118]:
print(len(sim.people))
print(len(sim.people._uid_map))
print(sim.people._uid_map)

12
13
array([9223372036854775807,                   0,                   1,
                         2,                   3,                   4,
                         5,                   6,                   7,
                         8,                   9,                  10,
                        11])
